# 下載youtube播放清單成mp4

In [3]:
from pytube import Playlist
from moviepy.editor import *
from pydub import AudioSegment
from pydub.utils import make_chunks
from pydub import AudioSegment
import os, re
import random
import math
import time
import pandas as pd

# 下載youtube播放清單成mp4
def download(p):

    # 撥放清單標題
    print(p.title)
    
    n=0
    nlist = []
    namelist = []
    urllist = p.video_urls
    
    if not os.path.isdir('./songinfo'):
        os.mkdir('./songinfo')
        
    # 逐一處理撥放清單中的影片
    for video in p.videos:
        try:
            progMP4 = video.streams.filter(progressive=True, file_extension='mp4')   # 設定下載方式及格式
            targetMP4 = progMP4.order_by('resolution').desc().first()  # 由畫質高排到低選畫質最高的來下載
            n = n+1 #下一部影片
            name = '%s%s.mp4' % (p.title,n)  # 給下載影片名
#             video_file = targetMP4.download(output_path='./musicfile/mp4/%s' %(p.title) , filename=name) # 下載影片並給輸出路徑
            namelist.append(video.title)
            nlist.append('%s%s' %(p.title,n))
        except: #有錯誤時跳過
            print('%sError' %(n))
            pass    
    nameurldf = pd.DataFrame((zip(nlist,namelist, urllist)), columns = ['songid','videoname','url'])
    nameurldf.to_csv('./songinfo/songinfo_%s.csv' %(p.title), index=False)


# mp4轉檔成wav
def videotowav(p):
    
    dir = './musicfile/mp4/%s' %(p.title)   # 給檔案來源路徑
    parentfolder = './musicfile/wav'        # 給檔案輸出目錄的上一層目錄
    outputpath = './musicfile/wav/%s' %(p.title)   # 給檔案輸出目錄

    # 如果沒有資料夾建立資料夾
    if not os.path.isdir(parentfolder):
        os.mkdir(parentfolder)
    if not os.path.isdir(outputpath):
        os.mkdir(outputpath)


    initial_count = 1  # 為了下面range()取到檔案總數+1，才有取到全部

    for path in os.listdir(dir):   # 找出目錄路徑內的所有資料夾及檔案
        if os.path.isfile(os.path.join(dir, path)):    # 如果是檔案就抓出來
            initial_count += 1              # 計算總共有幾個檔案+1


    for n in range(1,initial_count):    # 從第一個到總數的檔案，一個一個轉檔
    # for n in range(1,2):
    # for n in os.listdir(dir):
        # 給mp4檔案來源名稱和路徑
        mp4_file = dir+'/%s%s.mp4' %(p.title,n)
        # 給wav檔案輸出名稱和路徑
        wav_file = outputpath+'/%s%s.wav' %(p.title,n)
        
        videoclip = VideoFileClip(mp4_file)   # 先匯入影檔
        audioclip = videoclip.audio           # 從影檔取出音檔
    #   audioclip.nchannels = 1    # 設定聲道數
        audioclip.write_audiofile(wav_file, fps=44000)  # 將取出的音檔寫入剛給的wav名稱及路經，並設採樣率44000
        audioclip.close()
        videoclip.close()    

# wav分割成30s
def split30s(p):

    dir = './musicfile/wav/%s' %(p.title)
    parentfolder = './musicfile/30s'
    outputpath = './musicfile/30s/%s' %(p.title)

    # 如果沒有資料夾建立資料夾
    if not os.path.isdir(parentfolder):
        os.mkdir(parentfolder)
    if not os.path.isdir(outputpath):
        os.mkdir(outputpath)

    # 迴圈目錄下所有檔案
    for each in os.listdir(dir):
        
        filename = re.findall(r"(.*?)\.wav", each) # 取出.wav字尾的檔名
        print(each)
        if each:
            # filename[0] += '.wav'
            # print(filename[0])

            wav = AudioSegment.from_file((dir+'/{}').format(each), "wav") # 開啟wav檔案
            size = 30000  # 切割的毫秒數 1s=1000

            chunks = make_chunks(wav, size)  # 將檔案切割為30s一塊
            numstop = len(chunks)-2
            for i, chunk in enumerate(chunks):    # 一段一段的chunk取出來
                if numstop>=i>=1:   # 不取前30s及最後30s的部分
#                 if i>=2 and len(chunk) >= 29999:   # 不取前60s及最後不到30s的部分
                    chunk_name = "{}-{}.wav".format(each.split(".")[0],i)
                    chunk.export((outputpath+'/{}').format(chunk_name), format="wav")

# wav分割成3s
def split3s(p):
    
    dir = './musicfile/wav/%s' %(p.title)
    parentfolder = './musicfile/3s'
    outputpath = './musicfile/3s/%s' %(p.title)

    # # 如果沒有資料夾建立資料夾
    if not os.path.isdir(parentfolder):
        os.mkdir(parentfolder)
    if not os.path.isdir(outputpath):
        os.mkdir(outputpath)

        # 迴圈目錄下所有檔案
    for each in os.listdir(dir):

        filename = re.findall(r"(.*?)\.wav", each) # 取出.wav字尾的檔名
        print(each)
        if each:
            # filename[0] += '.wav'
            # print(filename[0])

            wav = AudioSegment.from_file((dir+'/{}').format(each), "wav") # 開啟wav檔案
            size = 3000  # 切割的毫秒數 3s=3000

            chunks = make_chunks(wav, size)  # 將檔案切割為3s一塊
            numstop = len(chunks)-10
            for i, chunk in enumerate(chunks):
                if numstop>=i>=20:   # 不取前60s及最後30s的部分
                    chunk_name = "{}-{}.wav".format(each.split(".")[0],i)
                    chunk.export((outputpath+'/{}').format(chunk_name), format="wav")
                    
# 30s分割成3s 
def split30sto3s(p):
    dir = './musicfile/30s/%s' %(p.title)
    parentfolder = './musicfile/3s'
    outputpath = './musicfile/3s/%s' %(p.title)

    # # 如果沒有資料夾建立資料夾
    if not os.path.isdir(parentfolder):
        os.mkdir(parentfolder)
    if not os.path.isdir(outputpath):
        os.mkdir(outputpath)

        # 迴圈目錄下所有檔案
    for each in os.listdir(dir):

        filename = re.findall(r"(.*?)\.wav", each) # 取出.wav字尾的檔名
        print(each)
        if each:
            # filename[0] += '.wav'
            # print(filename[0])

            wav = AudioSegment.from_file((dir+'/{}').format(each), "wav") # 開啟wav檔案
            size = 3000  # 切割的毫秒數 1s=1000
            chunks = make_chunks(wav, size)  # 將檔案切割為30s一塊

            for i, chunk in enumerate(chunks):
                if len(chunk)>=2999:
                    chunk_name = "{}-{}.wav".format(each.split(".")[0],i)
                    chunk.export((outputpath+'/{}').format(chunk_name), format="wav")

In [4]:
blues = 'https://youtube.com/playlist?list=PL_4CNbRBv_2W-q07_FUed0CacrTEiQSf1&utm_source=EKLEiJECCKjOmKnC5IiRIQ'
classical = 'https://youtube.com/playlist?list=PLVC22euIAtW5qdIKJXnAeeLmBzB1_MVR2'
country = 'https://youtube.com/playlist?list=PLYHUBTqX1rAbsUo_WzulBdwkCsW2erVul&utm_source=EKLEiJECCKjOmKnC5IiRIQ'
disco = 'https://youtube.com/playlist?list=PLPC4yaVJHMyIGxUSW3C7V0IkNcCUAXjkV'
hiphop = 'https://youtube.com/playlist?list=PL_4CNbRBv_2Xpekdw3MP9VMSwnd2S2JOr&utm_source=EKLEiJECCKjOmKnC5IiRIQ'
jazz = 'https://youtube.com/playlist?list=PLVC22euIAtW7wSKKdD5gGxVLEjWCxGI-S'
metal = 'https://youtube.com/playlist?list=PLYHUBTqX1rAY47k8a0zYCVmU9-5MYfbOF&utm_source=EKLEiJECCKjOmKnC5IiRIQ'
pop = 'https://www.youtube.com/playlist?list=PLPC4yaVJHMyLCQmtlUrv2UpbhpqQ6CqUd'
reggae = 'https://www.youtube.com/playlist?list=PLMWnoVyYvwScuENn5hrmFizzSjNj6Nrc0'
rock = 'https://www.youtube.com/playlist?list=PLQW_s3feqm8U9d62sRd54-mVdRBlli5BM'
usermusictest = 'https://www.youtube.com/playlist?list=PLQW_s3feqm8Wbpn9PHoJE64ZPIm8KU5IC'

In [5]:
# 建立 Playlist 物件
p = Playlist(pop)

start = time.perf_counter()

download(p)
# videotowav(p)
# split30s(p)
# # split3s(p)
# split30sto3s(p)

print("This time is being calculated")

end = time.perf_counter()

print(end - start)

pop
This time is being calculated
74.56252319999999


In [6]:
df = pd.read_csv('./songinfo/songinfo_pop.csv')
df

,songid,videoname,url
0,pop1,Sasha Alex Sloan - Is It Just Me? (Lyric Video...,https://www.youtube.com/watch?v=nOa3hrRD9mk
1,pop2,Jeremy Zucker & Chelsea Cutler - This Is How Y...,https://www.youtube.com/watch?v=h8k1VpzM8kI
2,pop3,Astrid S - It´s Ok If You Forget Me (Lyrics),https://www.youtube.com/watch?v=hI4HND3ZG58
3,pop4,Peder Elias - When I'm Still Getting Over You ...,https://www.youtube.com/watch?v=FYhoRoEL-uk
4,pop5,Maximillian - Letters (Lyric Video),https://www.youtube.com/watch?v=q-iLZ4dn9d8
5,pop6,Charlie Puth - That's Hilarious (Lyrics),https://www.youtube.com/watch?v=Q_Ccocsf6zU
6,pop7,"Ariana Grande, Kid Cudi - Just Look Up (Lyrics)",https://www.youtube.com/watch?v=KxJNTVYJVdQ
7,pop8,Taylor Swift - Run (Taylor's Version) ft. Ed S...,https://www.youtube.com/watch?v=PwfgNmouxNA
8,pop9,"Surf Mesa, Madison Beer - Carried Away (666D A...",https://www.youtube.com/watch?v=RySD5gGmWlw
9,pop10,Isac Elliot - Waving At Cars (Lyrics),https://www.youtube.com/watch?v=M19MXcNX5QA


In [12]:
from pytube import YouTube
def downloadv(videourl,file_name):
    yt = YouTube(videourl)
    progMP4 = yt.streams.filter(progressive=True, file_extension='mp4')   # 設定下載方式及格式
    targetMP4 = progMP4.order_by('resolution').desc().first()  # 由畫質高排到低選畫質最高的來下載
    name = '%s.mp4' % (file_name)  # 給下載影片名
    video_file = targetMP4.download(output_path='./musicfile/mp4/%s' %(file_name) , filename=name) # 下載影片並給輸出路徑       

In [13]:
videourl = 'https://www.youtube.com/watch?v=tQ0yjYUFKAE&list=PLQW_s3feqm8Wbpn9PHoJE64ZPIm8KU5IC&index=2'
file_name = 'abc'
downloadv(videourl,file_name)